## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page: [Clinical Text Analysis](https://aws.amazon.com/marketplace/pp/prodview-v7nmnswuxgcue).
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Pipeline to Extract clinical entities, Assign Assertion Status and Find Relations

Overview:

Clinical entity detection, assertion status assignment, and relation extraction are crucial for analyzing medical texts. These processes can be incredibly useful to healthcare professionals, researchers, and medical NLP practitioners, enabling the extraction of valuable insights from clinical literature, electronic health records, and patient notes.


- **Model**: [en.explain_doc.clinical_granular](https://nlp.johnsnowlabs.com/2024/05/07/explain_clinical_doc_granular_en.html)
- **Model Description**: A pipeline designed for extracting clinical entities using the `ner_jsl` NER model, assigning their assertion status with the `assertion_jsl` model, and establishing relationships between the extracted entities with the `re_test_result_date` relation extraction model.


In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [ ]:
import json
import os
import boto3
import pandas as pd
import sagemaker as sage
from sagemaker import ModelPackage
from sagemaker import get_execution_role
from IPython.display import display
from urllib.parse import urlparse

In [ ]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [4]:
model_name = "en-explain-doc-clinical-granular"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.2xlarge"

## 2. Create a deployable model from the model package.

In [5]:
model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker_session,
)

### Input Format

To use the model, you need to provide input in one of the following supported formats:

#### JSON Format

Provide input as JSON. We support two variations within this format:

1. **Array of Text Documents**: 
   Use an array containing multiple text documents. Each element represents a separate text document.

   ```json
   {
       "text": [
           "Text document 1",
           "Text document 2",
           ...
       ]
   }

    ```

2. **Single Text Document**:
   Provide a single text document as a string.


   ```json
    {
        "text": "Single text document"
    }
   ```

#### JSON Lines (JSONL) Format

Provide input in JSON Lines format, where each line is a JSON object representing a text document.

```
{"text": "Text document 1"}
{"text": "Text document 2"}
```

## 3. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### A. Deploy the SageMaker model to an endpoint

In [ ]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
)

Once endpoint has been created, you would be able to perform real-time inference.

In [7]:
def invoke_realtime_endpoint(record, content_type="application/json", accept="application/json"):
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=json.dumps(record) if content_type == "application/json" else record,
    )

    response_body = response["Body"].read().decode("utf-8")

    if accept == "application/json":
        return json.loads(response_body)
    elif accept == "application/jsonlines":
        return response_body
    else:
        raise ValueError(f"Unsupported accept type: {accept}")

### Initial Setup

In [8]:
docs = [
    '''The patient is a 21-day-old Caucasian male here for 2 days of congestion - mom has been suctioning yellow discharge from the patient's nares, plus she has noticed some mild problems with his breathing while feeding (but negative for any perioral cyanosis or retractions). Additionally, there is no side effect observed after Influenza vaccine. One day ago, mom also noticed a tactile temperature and gave the patient Tylenol. Baby also has had some decreased p.o. intake. His normal breast-feeding is down from 20 minutes q.2h. to 5 to 10 minutes secondary to his respiratory congestion. He sleeps well, but has been more tired and has been fussy over the past 2 days. The parents noticed no improvement with albuterol treatments given in the ER. His urine output has also decreased; normally he has 8 to 10 wet and 5 dirty diapers per 24 hours, now he has down to 4 wet diapers per 24 hours. Mom denies any diarrhea. His bowel movements are yellow colored and soft in nature.''',
    '''The patient is a 40-year-old white male who presents with a chief complaint of "chest pain". The patient is diabetic and has a prior history of coronary artery disease. The patient presents today stating that his chest pain started yesterday evening and has been somewhat intermittent. He has been advised Aspirin 81 milligrams QDay, insulin 50 units in a.m. HCTZ 50 mg QDay. Nitroglycerin 1/150 sublingually PRN chest pain.'''
]


sample_text = "The patient admitted for gastrointestinal pathology, under working treatment. History of prior heart murmur with echocardiogram findings as above on March 1998. According to the latest echocardiogram, basically revealed normal left ventricular function with left atrial enlargement . Based on the above findings, we will treat her medically with ACE inhibitors and diuretics and see how she fares."

### JSON

In [9]:
input_json_data = {"text": sample_text}
data = invoke_realtime_endpoint(input_json_data, content_type="application/json", accept="application/json")

ner_df = pd.DataFrame(data[0]["ner_predictions"])
assertion_df = pd.DataFrame(data[0]["assertion_predictions"])
relation_df = pd.DataFrame(data[0]["relation_predictions"])

In [10]:
ner_df

,ner_chunk,begin,end,ner_label,ner_confidence
0,admitted,12,19,Admission_Discharge,0.9993
1,gastrointestinal pathology,25,50,Clinical_Dept,0.7198
2,heart murmur,95,106,Heart_Disease,0.5289
3,echocardiogram,113,126,Test,0.9935
4,March 1998,149,158,Date,0.874
5,echocardiogram,185,198,Test,0.9402
6,normal,220,225,Test_Result,0.4029
7,left ventricular function,227,251,Test,0.34836665
8,left atrial enlargement,258,280,Heart_Disease,0.5319666
9,her,327,329,Gender,0.9992


In [11]:
assertion_df

,ner_chunk,begin,end,ner_label,assertion,assertion_confidence
0,heart murmur,95,106,Heart_Disease,Past,1.0
1,echocardiogram,113,126,Test,Past,1.0
2,echocardiogram,185,198,Test,Past,0.9729
3,normal,220,225,Test_Result,Present,1.0
4,left ventricular function,227,251,Test,Past,1.0
5,left atrial enlargement,258,280,Heart_Disease,Present,1.0
6,ACE inhibitors,346,359,Drug_Ingredient,Planned,1.0
7,diuretics,365,373,Drug_Ingredient,Hypothetical,0.999


In [12]:
relation_df

,ner_chunk1,ner_chunk1_begin,ner_chunk1_end,ner_label1,ner_chunk2,ner_chunk2_begin,ner_chunk2_end,ner_label2,relations,relation_confidence
0,heart murmur,95,106,Heart_Disease,echocardiogram,113,126,Test,is_finding_of,1.0
1,heart murmur,95,106,Heart_Disease,March 1998,149,158,Date,is_date_of,1.0
2,echocardiogram,113,126,Test,March 1998,149,158,Date,is_date_of,1.0
3,echocardiogram,185,198,Test,normal,220,225,Test_Result,O,0.9999862
4,echocardiogram,185,198,Test,left atrial enlargement,258,280,Heart_Disease,is_finding_of,1.0
5,normal,220,225,Test_Result,left ventricular function,227,251,Test,is_result_of,1.0
6,left ventricular function,227,251,Test,left atrial enlargement,258,280,Heart_Disease,is_finding_of,0.99999976


### JSON Lines

In [13]:
def create_jsonl(records):
    if isinstance(records, str):
        records = [records]
    json_records = [{"text": text} for text in records]
    json_lines = "\n".join(json.dumps(record) for record in json_records)
    return json_lines

In [14]:
input_jsonl_data = create_jsonl(sample_text)
data = invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"ner_predictions": [{"ner_chunk": "admitted", "begin": 12, "end": 19, "ner_label": "Admission_Discharge", "ner_confidence": "0.9993"}, {"ner_chunk": "gastrointestinal pathology", "begin": 25, "end": 50, "ner_label": "Clinical_Dept", "ner_confidence": "0.7198"}, {"ner_chunk": "heart murmur", "begin": 95, "end": 106, "ner_label": "Heart_Disease", "ner_confidence": "0.5289"}, {"ner_chunk": "echocardiogram", "begin": 113, "end": 126, "ner_label": "Test", "ner_confidence": "0.9935"}, {"ner_chunk": "March 1998", "begin": 149, "end": 158, "ner_label": "Date", "ner_confidence": "0.874"}, {"ner_chunk": "echocardiogram", "begin": 185, "end": 198, "ner_label": "Test", "ner_confidence": "0.9402"}, {"ner_chunk": "normal", "begin": 220, "end": 225, "ner_label": "Test_Result", "ner_confidence": "0.4029"}, {"ner_chunk": "left ventricular function", "begin": 227, "end": 251, "ner_label": "Test", "ner_confidence": "0.34836665"}, {"ner_chunk": "left atrial enlargement", "begin": 258, "end": 280, "ner_la

### B. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Batch inference

In [16]:
validation_json_file_name = "input.json"
validation_jsonl_file_name = "input.jsonl"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/jsonl/"

def upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/{file_format[1:]}/{file_name}",
        Body=input_data.encode("UTF-8"),
    )

In [17]:
# Create JSON and JSON Lines data
input_jsonl_data = create_jsonl(docs)
input_json_data = json.dumps({"text": docs})

# Upload JSON and JSON Lines data to S3
upload_to_s3(input_json_data, validation_json_file_name)
upload_to_s3(input_jsonl_data, validation_jsonl_file_name)

### JSON

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [19]:
def retrieve_json_output_from_s3(validation_file_name):
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    display(data)

In [20]:
retrieve_json_output_from_s3(validation_json_file_name)

[{'ner_predictions': [{'ner_chunk': '21-day-old',
    'begin': 17,
    'end': 26,
    'ner_label': 'Age',
    'ner_confidence': '0.997'},
   {'ner_chunk': 'Caucasian',
    'begin': 28,
    'end': 36,
    'ner_label': 'Race_Ethnicity',
    'ner_confidence': '0.9995'},
   {'ner_chunk': 'male',
    'begin': 38,
    'end': 41,
    'ner_label': 'Gender',
    'ner_confidence': '0.9998'},
   {'ner_chunk': '2 days',
    'begin': 52,
    'end': 57,
    'ner_label': 'Duration',
    'ner_confidence': '0.805'},
   {'ner_chunk': 'congestion',
    'begin': 62,
    'end': 71,
    'ner_label': 'Symptom',
    'ner_confidence': '0.9049'},
   {'ner_chunk': 'mom',
    'begin': 75,
    'end': 77,
    'ner_label': 'Gender',
    'ner_confidence': '0.9907'},
   {'ner_chunk': 'suctioning yellow discharge',
    'begin': 88,
    'end': 114,
    'ner_label': 'Symptom',
    'ner_confidence': '0.26813334'},
   {'ner_chunk': 'nares',
    'begin': 135,
    'end': 139,
    'ner_label': 'External_body_part_or_region',


### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [22]:
def retrieve_jsonlines_output_from_s3(validation_file_name):

    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

In [23]:
retrieve_jsonlines_output_from_s3(validation_jsonl_file_name)

{"ner_predictions": [{"ner_chunk": "21-day-old", "begin": 17, "end": 26, "ner_label": "Age", "ner_confidence": "0.997"}, {"ner_chunk": "Caucasian", "begin": 28, "end": 36, "ner_label": "Race_Ethnicity", "ner_confidence": "0.9995"}, {"ner_chunk": "male", "begin": 38, "end": 41, "ner_label": "Gender", "ner_confidence": "0.9998"}, {"ner_chunk": "2 days", "begin": 52, "end": 57, "ner_label": "Duration", "ner_confidence": "0.805"}, {"ner_chunk": "congestion", "begin": 62, "end": 71, "ner_label": "Symptom", "ner_confidence": "0.9049"}, {"ner_chunk": "mom", "begin": 75, "end": 77, "ner_label": "Gender", "ner_confidence": "0.9907"}, {"ner_chunk": "suctioning yellow discharge", "begin": 88, "end": 114, "ner_label": "Symptom", "ner_confidence": "0.26813334"}, {"ner_chunk": "nares", "begin": 135, "end": 139, "ner_label": "External_body_part_or_region", "ner_confidence": "0.7284"}, {"ner_chunk": "she", "begin": 147, "end": 149, "ner_label": "Gender", "ner_confidence": "0.9978"}, {"ner_chunk": "mil

In [ ]:
model.delete_model()

### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

